# Neutron Media

The Neutron Media in TRIPOLI-5 contains all the nuclear data needed by compositions and collision physics models, which are necessary for performing a simulation.

In this notebook, we will demonstrate how to configure and build a Neutron Media object using the `tripoli5.materials.NeutronMediumBuilder` class. Once the `tripoli5.materials.NeutronMedia` object is built, it can be transmitted to the [simulation](simulation.ipynb).

In order to build a Neutron Media object, the only two mandatory data are:

- A **catalog** indexing nuclear data paths (e.g. neutron data, thermal scattering laws, ...).
- A list of [**mixtures**](mixtures.ipynb) defining the material compositions.

Optionally, the user may configure through the builder several parameters of the particle physics modeling such as for instance a **lower bound energy cut**, a **reactions used in the transport**, or the method for handling the **Unresolved Resonance Range**

These options are set using the `NeutronMediumBuilder` class which allows for checking consistency of the various options and then build a neutron media object.

To configure the `NeutronMedia`, the `NeutronMediumBuilder` class needs to be instantiated with its constructor and can then be configured with the following methods:  
**Mandatory Methods:**

- `withMixtures`: Specify the list of mixtures (material compositions).
- `withCatalog`: Specifies the nuclear data catalog.

**Optional Methods:**

- `withLowerEnergyCut`: Sets the simulation lower energy threshold.
- `withURR`: Selects the Unresolved Resonance Range options. Possible options are: "none", "ace", "calendf".
- `withDopplerBroadening`: Choose the Doppler Broadening options for the free gas data. Possible methods are: "PreTabulated", "Sigma1".
- `withSigma1Scheduler`: Select the SIGMA1 GPU processing options.
- `withEpithermalBroadening`: Set the Epithermal Broadening options.
- `withOrigin`: Specifies the source of the nuclear data. Possible sources are: "Directory", "Archive".
- `withEnergyGridParameters`: Specify the energy grid parameters. Possible search types are: "BinarySearch", "HashMap".
- `withCollisionSampler`: Specify the sampling method for collisions. Possible sampling types are: "analog", "implicit_capture", "branchless".
- `withTemperatureInterpolationMethod`: Specify the temperature interpolation method. This method applies to free gas, thermal scattering and URR data in "PreTabulated" mode (see `withDopplerBroadening` option) and only to thermal scattering data in "Sigma1" mode. Thus, when applying  "Sigma1" algorithm, this parameter will be unused if no thermal data are involved (note that the combination of "Sigma1" algorithm and URR treatment is not available yet). Possible methods are: "nearest", "linear", "sqrt".

There are several methods for sampling the outgoing particles of a collision. Currently, one can choose between _analog sampling_, _nuclide-wise implicit capture_ or _nuclide-wise branchless sampling_. The default is analog sampling; all reaction channels can be sampled and the reaction yield is used to determine the number of outgoing particles
.
In the following, we will depict in detail each option starting with building the "all-defaults" neutron media. 

## Prerequisites: imports, Catalog and Mixtures 

First, let's import TRIPOLI-5 modules and necessary classes.

### Imports

In [1]:
%load_ext wurlitzer

In [2]:
import tripoli5
from tripoli5.core.literals import K, MeV, eV

### Catalog
The path to nuclear/mass data needs to be specified through the `tripoli5.delos` module.  
Either by using an **environment variable**: `T5_CATALOG_PATH`, or by **specifying a file path directly**.

In [3]:
catalog = tripoli5.delos.Catalog.fromFile(
    f"/opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/endfb8r0_mcnp_v{tripoli5.delos.CATALOG_VERSION}.yaml"
)

[2025-09-10 16:28:42.893] [info] Loading catalog from file: /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/endfb8r0_mcnp_v2.2.0.yaml


### Mixtures
In order to create a neutron media, we first need to create [mixtures](mixtures.ipynb), i.e., material compositions, using the `MixtureBuilder`.

In [4]:
reactor = (
    tripoli5.materials.MixtureBuilder("concentrations")
    .withName("reactor")
    .withCatalog(catalog)
    .withTemperature(294.0 * K)
    .add(
        ["U234", "U235", "U236", "U238"],
        [4.8416e-05, 4.8151e-03, 1.7407e-05, 4.3181e-02],
    )
    .build()
)

reflector = (
    tripoli5.materials.MixtureBuilder("concentrations")
    .withName("reflector")
    .withCatalog(catalog)
    .withTemperature(294.0 * K)
    .add(
        ["U234", "U235", "U236", "U238"],
        [2.8672e-07, 1.0058e-04, 1.1468e-06, 4.7677e-02],
    )
    .build()
)

mixtures = [reactor, reflector]

[2025-09-10 16:28:42.910] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/mass/njoy_ace/v0.2.0/archive/wrapped_mass_data.bin
[2025-09-10 16:28:42.912] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/mass/njoy_ace/v0.2.0/archive/wrapped_mass_data.bin


## Neutron Media with all default options

Defining a `NeutronMedia` with all default options, requiring only the mixtures and a catalog, is performed as below.  
Note that building the media can be done either using a single liner or explicitely instantiating the builder and then building the media.  
The two approaches are described here. For the rest of this notebook, only the first version will be used. 

In [5]:
# one liner syntax
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .build()
)

# explicitely instantiating the builder
builder = tripoli5.materials.NeutronMediumBuilder()
builder.withMixtures(mixtures)
builder.withCatalog(catalog)
media = builder.build()

[2025-09-10 16:28:42.919] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:42.920] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:42.924] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:42.924] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:42.924] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:42.925] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:42.951] [info]   Building free gas colli

## Lower energy threshold

The lower energy limit of the simulation can be set as below taking as argument an `Energy` value. 
The default value is $1.0 10^{-11}$ MeV.  

In [6]:
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withLowerEnergyCut(lower_bound=1.0e-5 * MeV)
    .build()
)

[2025-09-10 16:28:43.069] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.070] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.074] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.074] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.074] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.074] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.089] [info]   Building free gas colli

## Unresolved Resonance Range treatment

The Unresolved Resonance Range (URR) refers to the energy region where the resonance structure of a nuclide is too complex to be resolved individually. In this region, statistical methods are used to describe the average behavior of the resonances.

Three options are available for treating the unresolved resonance range, specified with the `withURRParameters` method. 

- `"none"`: No treatment for the URR. This is the default behavior.
- `"ace"`: probability tables from ACE formatted files (usually processed using [NJOY](https://github.com/njoy)'s module PURR).
- `"calendf"`: probability tables processed with the [CALENDF](https://www.oecd-nea.org/tools/abstract/detail/nea-1278/) software. 


To select the *CALENDF* treatment for the URR, you can use the following code:



In [7]:
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withURR(
        parameter="calendf"
    )  # here "none", "ace" or "calendf" may be used as parameter
    .build()
)

[2025-09-10 16:28:43.124] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.125] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.130] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/urr/calendf/v0.2.0/dir/U234.294.urr.calendf.bin
[2025-09-10 16:28:43.133] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.133] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.133] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.133] [info] Loading /opt/tripol

## Doppler Broadening of freegas cross section

Cross sections at a given temperature are usually computed before calculation by an external preprocessing of nuclear data software. Cross sections are tabulated at several specific temperature. When the desired temperature is not available or if lots of temperature are required, it may be inconvenient/unfeasable to process tabulated cross section for every required temperatures. In order to circumvent this issue, TRIPOLI-5 allows the use of an on-the-fly Doppler Broadening of cross sections. Currently, this is achieved using the SIGMA1 algorithm using tabulated cross section at a specific **base** temperature.  

**Warning:** Note that the SIGMA1 algorithm applies only to freegas cross sections. **Currently**, it cannot be applied with a specific URR treatment. However, we can combine SIGMA1 algorithm and stochastic interpolation for the thermal scattering data. 

This option can be selected using the `withDopplerBroadening` method with the following parameters: 

- Doppler Broadening Method: `"PreTabulated"` (default) / `"Sigma1"`
- Base Temperature: Optionnal with the SIGMA1 method (Default is $0.0 K$)
- Doppler Threshold: Optionnal with the SIGMA1 method (Default is $1.0 MeV$)

 Below is a syntax example of turning on the SIGMA1 Doppler Broadening:

In [8]:
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withDopplerBroadening(
        doppler_broadening_method="Sigma1",
        base_temperature=294.0 * K,
        max_energy=20.0e-3 * MeV,
    )
    .build()
)

# below is the pretabulated version which is equivalent to NOT specify the Doppler Broadening option
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withDopplerBroadening("PreTabulated")
    .build()
)

[2025-09-10 16:28:43.195] [info] Building U234 for SIGMA1 Doppler broadening
[2025-09-10 16:28:43.195] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.199] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.199] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.199] [info] Building U235 for SIGMA1 Doppler broadening
[2025-09-10 16:28:43.200] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.215] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.215] [info]   Building reaction

### GPU processing of the SIGMA1 algorithm

Due to its intensive computation effort, the SIGMA1 algorithm is a good candidate to be ported on GPU.  
To that end, can set some parallelization options for Sigma1 cross-section computation using the `withSigma1Scheduler` method. By **default**, if this method is not called, there is **no GPU parallelization for Sigma1**.

The `withSigma1Scheduler` method argument is a `tripoli5.materials.Sigma1SchedulerOptions` object containing all parameters regarding the GPU ported version of the SIGMA1. 
The arguments for the `tripoli5.materials.Sigma1SchedulerOptions` object are:

- **target**: Specifies the target platform for parallelization. Options include:  

        - `"None"`: No parallelization.  
        - `"Host"`: Parallelization on the host CPU.  
        - `"Cuda"`: Parallelization using NVIDIA CUDA.  
        - `"OpenACC"`: Parallelization using OpenACC.  
- **nb_threads**: The number of threads to be used for parallelization. Note that the number of threads defined **must be consistent with the number defined in the simulation**.
- **gpu_rate**: The rate at which GPU resources are utilized. This is a value between 0 and 1, where 0 means no GPU usage and 1 means full GPU usage.
- **offload_size**: The number of Sigma1 computation tasks to be offloaded to the GPU for processing. A computation task is a triplet consisting of a nuclide, energy particle, and material temperature.
- **policy**: The scheduling policy for parallelization. Options include:  

        - `"Static"`: Static scheduling, where tasks are evenly distributed among threads.  
        - `"Dynamic"`: Dynamic scheduling, where tasks are dynamically assigned to threads as they become available.  

In the example below, the Sigma1 scheduler is configured to use CUDA for parallelization with 20 threads, a GPU rate of 0.95, an offload size of 1000 computation tasks and a dynamic scheduling policy.

In [9]:
opts = tripoli5.materials.Sigma1SchedulerOptions(
    target="Cuda",
    nb_threads=20,
    gpu_rate=0.95,
    offload_size=1000,
    policy="Dynamic",
)

media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withDopplerBroadening(
        doppler_broadening_method="Sigma1",
        base_temperature=294.0 * K,
        max_energy=20.0e-3 * MeV,
    )
    .withSigma1Scheduler(opts)
    .build()
)

[2025-09-10 16:28:43.315] [info] Building U234 for SIGMA1 Doppler broadening
[2025-09-10 16:28:43.316] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.320] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.320] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.320] [info] Building U235 for SIGMA1 Doppler broadening
[2025-09-10 16:28:43.321] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.337] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.337] [info]   Building reaction

## Epithermal broadening of the elastic scattering kernel

In neutron interactions with matter, relative velocity effects between the incident neutron and the collided nucleus at thermal equilibrium must be considered. The thermal agitation of nuclei results in Doppler-broadened cross sections and scattering kernels. In this section, we described the options for handling the scattering kernels.

Two models are available for taking into account the Doppler broadening of elastic scattering kernels: 

- the Sampling of the Velocity of the Target nucleus (SVT)
- the Doppler Broadening Rejection Correction (DBRC)
  
These methods can be applied to all nuclides or to a subset of nuclides.
The SVT model is valid for light nuclei but breaks down for heavy resonant nuclei like $^{235}U$. For such nuclei, the DBRC model is recommended.

The selection of the method and its parameters is performed using the `withEpithermalBroadening` method. This method takes as an argument an `EpithermalBroadeningParameters` object defined below. 

The option selection mechanism is quite flexible and allow mixing models according to the energy range for each nuclide. This flexibility comes at a cost of complexity when defining these options though the use of a `NuclideSelector`. Thus, we chose, for this notebook, to expose the most common use cases before moving to a complete description of the available options.  

### Default options

The default model in TRIPOLI-5 for Doppler broadening of elastic scattering kernels is the SVT (Sampling of the Velocity of the Target nucleus) model. This model assumes nuclei are at thermal equilibrium with a Maxwell-Boltzmann velocity distribution and derives scattered neutron parameters from two-body kinematics.

TRIPOLI-5 uses the SVT model in the region from $10^{−5}$ eV to an upper threshold energy $E_\mathrm{threshold} = 400k_BT$, where $k_B = 8.617\times10^{-5}$ [eV/K] is the Boltzmann constant, and $T$ is the medium temperature. Above $E_\mathrm{threshold}$, the asymptotic scattering kernel is used.

If thermal scattering laws are available in the thermal region (typically between $10^{−5}$ eV and $E_\mathrm{therm}$, with $E_\mathrm{therm}$ around 5 eV), TRIPOLI-5 uses these thermal sections up to $E_\mathrm{therm}$. Above $E_\mathrm{therm}$, it chooses the SVT or the asymptotic elastic kernel, based on whether the incident neutron energy is below or above $E_\mathrm{threshold}$, respectively.

Although not required, one can explicitely specify the default Epithermal Doppler broadening treatment with the static method `EpithermalBroadeningParameters.defaultParams()` class. 

In [10]:
default_params = tripoli5.materials.EpithermalBroadeningParameters.defaultParams()
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withEpithermalBroadening(default_params)
    .build()
)

# which is equivalent to specifying nothing
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .build()
)

[2025-09-10 16:28:43.372] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.373] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.377] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.377] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.377] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.378] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.393] [info]   Building free gas colli

### Turning off the Epithermal broadening treatment

It is possible to turn off the Epithermal broadening treatment for all nuclides by using the following syntax: 

In [11]:
no_epithermal_params = tripoli5.materials.EpithermalBroadeningParameters.empty()
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withEpithermalBroadening(no_epithermal_params)
    .build()
)

[2025-09-10 16:28:43.479] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.479] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.483] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.483] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.483] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.484] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.500] [info]   Building free gas colli

### Applying one model to all nuclides  

In order to apply one option ("None", "SVT" or "DBRC") to all nuclides of the system over the entire energy range from the lower bound of the simulation to $400k_BT$, one can use the syntax below.  
The example below shows the selection of the "DBRC" option which can be replaced by "None" for no treatment or "SVT" for the SVT treatment. 

In [12]:
# apply over the entire energy range
range_dbrc = tripoli5.materials.EnergyRange(
    None, tripoli5.materials.Threshold.temperature_dependent(400)
)
# select all nuclides
selection_all_nuclides = tripoli5.materials.NuclideSelector.all(energy_range=range_dbrc)
# apply dbrc for all nuclides over this range
epithermal_opts = tripoli5.materials.EpithermalBroadeningParameters.empty().add(
    method="DBRC", nuclide_selector=selection_all_nuclides
)
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withEpithermalBroadening(epithermal_opts)
    .build()
)

[2025-09-10 16:28:43.536] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.537] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.541] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.541] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.0.bin
[2025-09-10 16:28:43.546] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.546] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.547] [info] Loading /opt/tripoli-5/delo

### Applying a model (different from the default one) to some nuclides  

If one wants to use the default parameters for most of the nuclides and "DBRC" (for instance or "None") for some of the nuclides (for instance the heavy ones), it is possible to select a specific treatment for a list of nuclides.

For instance in the example below, all nuclides uses the default SVT model except for the U235 and U238 which uses the DBRC method.  
**Warning:** the list of nuclides must be a Python [set](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset)

In [13]:
# apply over the entire energy range
range_dbrc = tripoli5.materials.EnergyRange(
    None, tripoli5.materials.Threshold.temperature_dependent(400)
)
# select only U235 and U238 for applying the DBRC
list_nuclides = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U235", "U238"]),  # the argument must be a Python set
    energy_range=tripoli5.materials.EnergyRange(
        None, tripoli5.materials.Threshold.temperature_dependent(400)
    ),
)
# apply SVT as the default behavior but DBRC for nuclides selected at the previous line
epithermal_opts = tripoli5.materials.EpithermalBroadeningParameters.defaultParams().add(
    method="DBRC", nuclide_selector=list_nuclides
)
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withEpithermalBroadening(epithermal_opts)
    .build()
)

[2025-09-10 16:28:43.727] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.728] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.732] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.732] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.732] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.733] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.750] [info]   Building free gas colli

### Changing the upper limit of the Epithermal broadening treatment 

Usually the Epithermal broadening treatment is applied up to $400k_BT$, if one wants to apply up to a different limit, it may be changed using the range definition. 

For instance, below, we impose the SVT model between the lower energy limit up to $300k_BT$:

In [14]:
# apply over the energy range up to 300 k_b T
range_dbrc = tripoli5.materials.EnergyRange(
    None, tripoli5.materials.Threshold.temperature_dependent(300)
)
# select all nuclides
selection_all_nuclides = tripoli5.materials.NuclideSelector.all(energy_range=range_dbrc)
# apply dbrc for all nuclides over this range
epithermal_opts = tripoli5.materials.EpithermalBroadeningParameters.empty().add(
    method="SVT", nuclide_selector=selection_all_nuclides
)
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withEpithermalBroadening(epithermal_opts)
    .build()
)

[2025-09-10 16:28:43.918] [info] Building U234 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.918] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U234.294.bin
[2025-09-10 16:28:43.922] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:43.922] [info]   Building reaction MTs = 20, 79, 21, 80, 66, 37, 51, 70, 71, 72, 74, 77, 73, 102, 89, 91, 67, 38, 90, 88, 87, 57, 86, 58, 54, 83, 60, 2, 61, 62, 63, 64, 65, 55, 84, 68, 56, 85, 69, 59, 53, 82, 16, 75, 17, 76, 19, 78, 81, 52
[2025-09-10 16:28:43.922] [info] Building U235 at temperatures: TemperatureRange(294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C])
[2025-09-10 16:28:43.924] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/U235.294.bin
[2025-09-10 16:28:43.942] [info]   Building free gas colli

### Custom selection of options

Now that we have seen how to handle the most common use cases, we will describe in more details the selection mechanism for applying the various methods.  

First, it is important to explicit that the `EpithermalBroadeningParameters` is built as a [stack](https://en.wikipedia.org/wiki/Stack_(abstract_data_type)) of `NuclideSelector`s associated to a method. A `NuclideSelector` being a pair consisting of a list of nuclides and an energy range. 

We will describe the behavior using an example. The nuclides in the mixtures we have used throughout this example are "U234", "U235", "U236" and "U238".  
Let's assume that we want to use the following options: 

- No model as the default behaviour (here it will concern "U234" and "U236")
- for U235 :
    - SVT between $10^{-5}$ eV and $1$ eV
    - DBRC between $5$ eV and $22$ eV
    - default behavior otherwise
- for U238:
    - DBRC between $10^{-5}$ eV and $3$ eV
    - DBRC between $5$ eV and $22$ eV
    - default behavior otherwise

To apply these options we will have to create 3 `NuclideSelector`s:

1. (U235, U238) on the range $5$ eV and $22$ eV
2. (U235) on the range $10^{-5}$ eV and $1$ eV
3. (U238) on the range $10^{-5}$ eV and $3$ eV

In [15]:
selector_1 = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U235", "U238"]),
    energy_range=tripoli5.materials.EnergyRange(
        tripoli5.materials.Threshold.energy(5 * eV),
        tripoli5.materials.Threshold.energy(22 * eV),
    ),
)
selector_2 = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U235"]),
    energy_range=tripoli5.materials.EnergyRange(
        tripoli5.materials.Threshold.energy(1e-5 * eV),
        tripoli5.materials.Threshold.energy(1 * eV),
    ),
)
selector_3 = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U238"]),
    energy_range=tripoli5.materials.EnergyRange(
        tripoli5.materials.Threshold.energy(1e-5 * eV),
        tripoli5.materials.Threshold.energy(3 * eV),
    ),
)

Now that selector have been defined, we will assign a method to each selector 

In [16]:
# choose method for each selector
epithermal_opts = (
    tripoli5.materials.EpithermalBroadeningParameters.empty()
    .add(method="DBRC", nuclide_selector=selector_1)
    .add(method="SVT", nuclide_selector=selector_2)
    .add(method="DBRC", nuclide_selector=selector_3)
)

It should be noted that the options works as a stack: when running the simulation, 
the collided isotope at a given energy is compared to the last added selector (`selector_3` in our example). If it matches then the method is applied, if it does not match then it is compared the second selector from the top (`selector_2` in our example) and so on until we reach the base one which is the default behavior. 
If two selectors overlap on some region, the last added method is chosen. 

For instance in the selectors below, the U235 applies the SVT method between $10^{-5}$ eV and $10$ eV and DBRC from $10$ eV to $22$ eV.

In [17]:
selector_1 = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U235", "U238"]),
    energy_range=tripoli5.materials.EnergyRange(
        tripoli5.materials.Threshold.energy(5 * eV),
        tripoli5.materials.Threshold.energy(22 * eV),
    ),
)
selector_2 = tripoli5.materials.NuclideSelector.from_list(
    list=set(["U235"]),
    energy_range=tripoli5.materials.EnergyRange(
        tripoli5.materials.Threshold.energy(1e-5 * eV),
        tripoli5.materials.Threshold.energy(10 * eV),
    ),
)
# choose method for each selector
epithermal_opts = (
    tripoli5.materials.EpithermalBroadeningParameters.empty()
    .add(method="DBRC", nuclide_selector=selector_1)
    .add(method="SVT", nuclide_selector=selector_2)
)

## Temperature interpolation method
 
The temperature interpolation method can be given for free gas data, thermal scattering data and URR data when Sigma1 is not applied (i.e. in PreTabulated mode) or for thermal scattering only in Sigma1 mode. It is set to `nearest` by default. 


In [18]:
# redefine mixtures involving thermal data
reactor = (
    tripoli5.materials.MixtureBuilder("concentrations")
    .withName("reactor")
    .withCatalog(catalog)
    .withTemperature(298.0 * K)
    .add(
        ["U234", "U235", "U236", "U238"],
        [4.8416e-05, 4.8151e-03, 1.7407e-05, 4.3181e-02],
    )
    .build()
)

reflector = (
    tripoli5.materials.MixtureBuilder("concentrations")
    .withName("reflector")
    .withCatalog(catalog)
    .withTemperature(298.0 * K)
    .add(
        ["H1_H2O"],
        [1e-03],
    )
    .build()
)

mixtures = [reactor, reflector]

media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withDopplerBroadening(
        doppler_broadening_method="Sigma1",
        base_temperature=294.0 * K,
        max_energy=20.0e-3 * MeV,
    )
    .withTemperatureInterpolationMethod("linear")
    .build()
)

# below is the pretabulated version. In such a case, the temperature interpolation method is used for free gas data as well since Sigma1 is not applied.
# Note also that in the pretabulated case the URR treatment can be switched on.
media = (
    tripoli5.materials.NeutronMediumBuilder()
    .withMixtures(mixtures)
    .withCatalog(catalog)
    .withDopplerBroadening("PreTabulated")
    .withTemperatureInterpolationMethod("linear")
    .withURR("ace")
    .build()
)

[2025-09-10 16:28:44.003] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/mass/njoy_ace/v0.2.0/archive/wrapped_mass_data.bin
[2025-09-10 16:28:44.005] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/mass/njoy_ace/v0.2.0/archive/wrapped_mass_data.bin
[2025-09-10 16:28:44.008] [info] Building H1_H2O for SIGMA1 Doppler broadening
[2025-09-10 16:28:44.009] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/freegas/njoy_ace/v0.11.0/dir/H1.294.bin
[2025-09-10 16:28:44.011] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/tsl/njoy_ace/v0.3.0/dir/H1_H2O.294.bin
[2025-09-10 16:28:44.021] [info] Loading /opt/tripoli-5/delos/nuclide/endfb8r0_mcnp/tsl/njoy_ace/v0.3.0/dir/H1_H2O.300.bin
[2025-09-10 16:28:44.353] [info]   Building free gas collisions at temperature 294 [K], 2.5334957256000003e-08 [MeV], 20.850000000000023 [°C]
[2025-09-10 16:28:44.353] [info]   Building reaction MTs = 102, 2
[2025-09-10 16:28:44.353] [info]   Building thermal collisions at tempera